# Use Case 3: Risk-Managed Portfolio

This notebook demonstrates:
1. External trades coming into the portfolio
2. Portfolio optimization to meet risk constraints
3. Factor exposure limits
4. Portfolio variance limits
5. Cost minimization subject to constraints

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from backtesting import Backtester, BacktestConfig, DataManager
from notebooks.notebook_utils import setup_plotting_style

%matplotlib inline
setup_plotting_style()

## Load Data

In [ ]:
data_manager = DataManager(data_dir='../sample_data', use_float32=True)
prices = data_manager.load_prices()

# Load factor model data
factor_exposures = data_manager.load_factor_exposures()
factor_returns = data_manager.load_factor_returns()

start_date = prices.index[0]
end_date = prices.index[-1]

print(f"Loaded {len(prices.columns)} securities")
print(f"Factor model: {len(factor_exposures.columns)} factors")
print(f"Date range: {start_date.date()} to {end_date.date()}")

## Option B: Generate External Trades Programmatically

For this demo, we'll generate random external trades with tags.

## Option A: Load External Trades from CSV

If you have external trades stored in CSV format, you can load them directly using DataManager.

In [ ]:
# Load external trades from CSV file
# CSV should have columns: date, ticker, qty, price, tag (optional)
# See sample_data/external_trades_example.csv for format

# Uncomment to load from CSV:
# trades_df = data_manager.load_external_trades('external_trades_example.csv')
# print(f"\nLoaded {len(trades_df)} trades from CSV")
# print(f"Date range: {trades_df['date'].min().date()} to {trades_df['date'].max().date()}")
# print(f"Tags: {trades_df['tag'].unique().tolist()}")
# print("\nSample trades:")
# print(trades_df.head())

# Convert to Use Case 3 format
# external_trades_by_date = data_manager.get_external_trades_by_date()
# print(f"\nConverted to Use Case 3 format: {len(external_trades_by_date)} dates")

# For this demo, we'll generate trades programmatically instead (see Option B below)

In [ ]:
# Generate random external trades with TAGS for counterparty attribution
# Use Case 3 expects: {ticker: [{'qty': X, 'price': Y, 'tag': 'optional_tag'}, ...]}
# Tags allow you to attribute PnL to different counterparties or groups

np.random.seed(42)
external_trades_by_date = {}

# Define counterparties
counterparties = ['Goldman Sachs', 'Morgan Stanley', 'JPMorgan', 'Citadel']

for date in prices.index[::5]:  # Every 5 days
    # Random subset of securities
    n_trades = np.random.randint(5, 20)
    trade_tickers = np.random.choice(prices.columns, n_trades, replace=False)
    
    trades = {}
    for ticker in trade_tickers:
        # Random trade size
        qty = np.random.randint(-1000, 1000)
        if qty != 0:
            # Get the close price for this ticker on this date
            close_price = prices.loc[date, ticker]
            
            # Randomly assign to a counterparty
            counterparty = np.random.choice(counterparties)
            
            # Format as list of dicts with TAG (required for Use Case 3)
            trades[ticker] = [{
                'qty': float(qty), 
                'price': float(close_price),
                'tag': counterparty  # Add tag for attribution
            }]
    
    if trades:
        external_trades_by_date[date] = trades

print(f"\nGenerated external trades for {len(external_trades_by_date)} days")
print(f"Counterparties: {counterparties}")
print(f"\nSample trade day: {list(external_trades_by_date.keys())[0].date()}")
sample_trades = list(external_trades_by_date.values())[0]
print(f"  Number of trades: {len(sample_trades)}")
print(f"\n  Sample trade with tag: {list(sample_trades.items())[0]}")

## Example 1: No Risk Constraints

First, run without risk constraints to see the baseline.

In [ ]:
inputs = {'external_trades': external_trades_by_date}

# Configure without constraints
config_no_constraints = BacktestConfig(
    initial_cash=10_000_000,
    max_adv_participation=0.05,
    risk_free_rate=0.02
)

print("Running WITHOUT risk constraints...")
backtester = Backtester(config_no_constraints, data_manager)
results_no_constraints = backtester.run(
    start_date=start_date,
    end_date=end_date,
    use_case=3,
    inputs=inputs,
    show_progress=True
)

results_no_constraints.print_summary()

## Example 2: With Factor Exposure Limits

In [ ]:
# Configure with factor exposure limits
config_factor_limits = BacktestConfig(
    initial_cash=10_000_000,
    max_adv_participation=0.05,
    max_factor_exposure={
        'Factor1': 0.1,
        'Factor2': 0.15,
        'Factor3': 0.1,
        'Factor4': 0.15,
        'Factor5': 0.1
    },
    risk_free_rate=0.02
)

print("\nRunning WITH factor exposure limits...")
backtester = Backtester(config_factor_limits, data_manager)
results_factor_limits = backtester.run(
    start_date=start_date,
    end_date=end_date,
    use_case=3,
    inputs=inputs,
    show_progress=True
)

results_factor_limits.print_summary()

## Example 3: With Portfolio Variance Limit

In [ ]:
# Configure with variance limit
config_variance_limit = BacktestConfig(
    initial_cash=10_000_000,
    max_adv_participation=0.05,
    max_portfolio_variance=0.0005,  # Variance limit
    risk_free_rate=0.02
)

print("\nRunning WITH portfolio variance limit...")
backtester = Backtester(config_variance_limit, data_manager)
results_variance_limit = backtester.run(
    start_date=start_date,
    end_date=end_date,
    use_case=3,
    inputs=inputs,
    show_progress=True
)

results_variance_limit.print_summary()

## Compare Results

In [ ]:
from notebooks.notebook_utils import compare_strategies

compare_strategies(
    [results_no_constraints, results_factor_limits, results_variance_limit],
    ['No Constraints', 'Factor Limits', 'Variance Limit'],
    metric='sharpe_ratio'
)

## Analyze Transaction Costs

The optimizer minimizes costs while meeting constraints.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (results, title) in enumerate([
    (results_no_constraints, 'No Constraints'),
    (results_factor_limits, 'Factor Limits'),
    (results_variance_limit, 'Variance Limit')
]):
    df = results.to_dataframe()
    cum_costs = df['transaction_cost'].cumsum()
    
    axes[idx].plot(df['date'], cum_costs, linewidth=2)
    axes[idx].set_title(title)
    axes[idx].set_ylabel('Cumulative Cost ($)')
    axes[idx].grid(True, alpha=0.3)
    
    total_cost = cum_costs.iloc[-1]
    axes[idx].text(0.05, 0.95, f'Total: ${total_cost:,.0f}',
                  transform=axes[idx].transAxes, va='top',
                  bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\nNote: Constrained portfolios may have higher costs due to rebalancing trades")

## Analyze Risk Over Time

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot volatility
for results, label in [
    (results_no_constraints, 'No Constraints'),
    (results_factor_limits, 'Factor Limits'),
    (results_variance_limit, 'Variance Limit')
]:
    df = results.to_dataframe()
    returns = df['daily_return']
    rolling_vol = returns.rolling(20).std() * np.sqrt(252)
    
    axes[0].plot(df['date'].iloc[20:], rolling_vol.iloc[20:], 
                label=label, linewidth=2, alpha=0.7)

axes[0].set_title('20-Day Rolling Volatility (Annualized)')
axes[0].set_ylabel('Volatility')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot net exposure
for results, label in [
    (results_no_constraints, 'No Constraints'),
    (results_factor_limits, 'Factor Limits'),
    (results_variance_limit, 'Variance Limit')
]:
    df = results.to_dataframe()
    axes[1].plot(df['date'], df['net_exposure'], 
                label=label, linewidth=2, alpha=0.7)

axes[1].set_title('Net Exposure')
axes[1].set_ylabel('Exposure ($)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Generate Reports

In [ ]:
results_factor_limits.generate_full_report(
    output_dir='../output/use_case_3_risk_managed',
    formats=['html', 'excel', 'csv']
)

print("Reports saved to ../output/use_case_3_risk_managed/")

In [ ]:
# Get PnL summary by counterparty/tag
pnl_summary = results_no_constraints.get_pnl_summary_by_tag()

print("=== PnL Attribution by Counterparty ===\n")
print(pnl_summary.to_string(index=False))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar chart of total PnL by counterparty
axes[0].bar(pnl_summary['tag'], pnl_summary['total_pnl'] / 1e6)
axes[0].set_title('Total PnL by Counterparty')
axes[0].set_xlabel('Counterparty')
axes[0].set_ylabel('Total PnL ($M)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# Sharpe ratio by counterparty
axes[1].bar(pnl_summary['tag'], pnl_summary['sharpe'])
axes[1].set_title('Sharpe Ratio by Counterparty')
axes[1].set_xlabel('Counterparty')
axes[1].set_ylabel('Sharpe Ratio')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Show cumulative PnL over time by counterparty
pnl_by_tag = results_no_constraints.get_pnl_by_tag()

if not pnl_by_tag.empty:
    fig, ax = plt.subplots(figsize=(15, 6))
    
    for tag in pnl_summary['tag']:
        tag_data = pnl_by_tag[pnl_by_tag['tag'] == tag]
        cumulative_pnl = tag_data['pnl'].cumsum()
        ax.plot(tag_data['date'], cumulative_pnl / 1e6, label=tag, linewidth=2, alpha=0.7)
    
    ax.set_title('Cumulative PnL by Counterparty Over Time')
    ax.set_xlabel('Date')
    ax.set_ylabel('Cumulative PnL ($M)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
print("\n=== Key Insights ===")
print(f"Best performing counterparty: {pnl_summary.iloc[0]['tag']}")
print(f"  Total PnL: ${pnl_summary.iloc[0]['total_pnl']:,.0f}")
print(f"  Sharpe Ratio: {pnl_summary.iloc[0]['sharpe']:.2f}")
print(f"  Win Rate: {pnl_summary.iloc[0]['win_rate']:.1%}")

## PnL Attribution by Counterparty

Now let's analyze PnL by counterparty using the tags we added to external trades.

## Summary

In this notebook, we explored Use Case 3:
1. ✓ External trades coming into the portfolio
2. ✓ Portfolio optimization to meet constraints
3. ✓ Factor exposure limits
4. ✓ Portfolio variance limits
5. ✓ Cost minimization

Key Takeaways:
- Optimizer finds minimum-cost solution to meet constraints
- Factor limits control exposure to risk factors
- Variance limits control overall portfolio risk
- Trade-off between costs and risk management
- Constrained portfolios may have higher transaction costs